In [32]:
import pandas as pd
import openpyxl
import os

# File Paths
data_file = r"C:\Users\Dell\Documents\NASSP-SOCU\NIN Integration & NSR Update Project\2043. OGUN\2043. OGUN\23897.SHAGAMU\hh.csv"
template_file = r"C:\Users\Dell\Documents\NASSP-SOCU\NIN Integration & NSR Update Project\Enumerator Tracking Sheet Template 2.xlsx"
output_folder = r"C:\Users\Dell\Documents\NASSP-SOCU\NIN Integration & NSR Update Project"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load the main data file (CSV)
df = pd.read_csv(data_file, dtype=str)  # Read as strings to preserve formatting

# Required columns mapping
column_mapping = {
    "nidhh": "NIDHH",
    "hhh": "NAME",
    "contactno": "PHONE NUMBER",
    "hhno": "OLD HOUSEHOLD NSR NO"
}

# Drop unnecessary columns and rename for consistency
df_filtered = df[["nidhh", "hhh", "contactno", "hhno", "ward_shp", "community", "lga_shp"]]
df_filtered.rename(columns=column_mapping, inplace=True)

# Get unique LGAs
lgas = df_filtered["lga_shp"].dropna().unique()

# Loop through each LGA
for lga in lgas:
    df_lga = df_filtered[df_filtered["lga_shp"] == lga]

    # Create LGA folder inside the output folder
    safe_lga = lga.translate(str.maketrans(r'\/:*?"<>|', "_________"))
    lga_folder = os.path.join(output_folder, safe_lga)
    os.makedirs(lga_folder, exist_ok=True)

    # Get unique wards in LGA
    wards = df_lga["ward_shp"].dropna().unique()

    for ward in wards:
        df_ward = df_lga[df_lga["ward_shp"] == ward]

        # Create a folder for this ward inside the LGA folder
        safe_ward = ward.translate(str.maketrans(r'\/:*?"<>|', "_________"))
        ward_folder = os.path.join(lga_folder, safe_ward)
        os.makedirs(ward_folder, exist_ok=True)

        # Get unique communities in the ward
        communities = df_ward["community"].dropna().unique()

        for community in communities:
            df_final = df_ward[df_ward["community"] == community]

            # Load the template
            wb = openpyxl.load_workbook(template_file)
            ws = wb.active  # Assuming first sheet

            # Format community name safely for filenames
            safe_community = community.translate(str.maketrans(r'\/:*?"<>|', "_________"))

            # Write LGA/Ward/Community to cell E3
            ws["E3"] = f"{lga[6:]}/{safe_ward[6:]}/{safe_community[8:]}"

            # Write data to the template
            for i, row in enumerate(df_final.itertuples(index=False, name=None), start=6):
                ws[f"B{i}"] = row[0]  # NIDHH
                ws[f"C{i}"] = row[1]  # NAME

                # ✅ Ensure "PHONE NUMBER" is blank instead of NaN
                phone_number = row[2] if pd.notna(row[2]) else ""
                if phone_number:
                    phone_number = str(phone_number).split(".")[0]  # Remove decimals
                    if phone_number.isdigit() and not phone_number.startswith("0"):
                        phone_number = "0" + phone_number
                ws[f"E{i}"] = phone_number

                # ✅ Copy "hhno" → "OLD HOUSEHOLD NSR NO" (Column D in template)
                old_household_nsr_no = row[3] if pd.notna(row[3]) else ""
                ws[f"D{i}"] = old_household_nsr_no

            # Save file inside the corresponding ward folder
            file_name = os.path.join(ward_folder, f"{safe_community}.xlsx")
            wb.save(file_name)
            wb.close()

print("✅ Export completed successfully! Files are organized by LGA, Ward, and Community.")


C:\Users\Dell\AppData\Local\Temp\ipykernel_4996\310261765.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns=column_mapping, inplace=True)


✅ Export completed successfully! Files are organized by LGA, Ward, and Community.
